In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata, getExternalDataDirectory as getMutperiodExternalDataDirectory
from mutperiodpy.quick_scripts.GetNRL import getNRL
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity, parseNucleosomeCountsDataForPlotting, plotPeriodicity

mutperiodHg19Directory = os.path.join(getMutperiodExternalDataDirectory(),"hg19")

liDataDirectory = os.path.join(getDataDir(), "Li_tXR-seq")

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

liRawCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
liNormalizedCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
NRLs = dict()

for nucleosomeCountsFilePath in getFilesInDirectory(liDataDirectory, DataTypeStr.rawNucCounts+".tsv"):

    if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
    else: radius = SINGLE_NUC

    countsMetadata = Metadata(nucleosomeCountsFilePath)

    liRawCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

for nucleosomeCountsFilePath in getFilesInDirectory(liDataDirectory, DataTypeStr.normNucCounts+".tsv"):

    if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
    else: radius = SINGLE_NUC

    countsMetadata = Metadata(nucleosomeCountsFilePath)

    liNormalizedCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

In [ ]:
for radius in liRawCounts:
    for nucleosomeMapName in liRawCounts[radius]:
        plot = parseAndPlotPeriodicity(liRawCounts[radius][nucleosomeMapName], dataCol = "Aligned_Strands_Counts",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Raw", yAxisLabel = "Counts")
        print(plot)

In [ ]:
for radius in liNormalizedCounts:
    for nucleosomeMapName in liNormalizedCounts[radius]:
        plot = parseAndPlotPeriodicity(liNormalizedCounts[radius][nucleosomeMapName], dataCol = "Normalized_Aligned_Strands",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Damage Normalized", yAxisLabel = "Normalized Counts")
        print(plot)